# Christmas Tree Packing - Baseline

Implement greedy placement with local search optimization following the seed prompt strategy.

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
from shapely.strtree import STRtree
import random
import math
from collections import defaultdict
import time

# Tree vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

print(f"Tree vertices: {len(TX)} points")
print(f"Tree height: {max(TY) - min(TY)}")
print(f"Tree width: {max(TX) - min(TX)}")

Tree vertices: 15 points
Tree height: 1.0
Tree width: 0.7


In [2]:
def create_tree_polygon(cx, cy, deg):
    """Create a tree polygon at position (cx, cy) with rotation deg degrees."""
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, deg, origin=(0, 0))
    poly = affinity.translate(poly, xoff=cx, yoff=cy)
    return poly

def has_overlap(poly1, poly2):
    """Check if two polygons overlap (not just touch)."""
    return poly1.intersects(poly2) and not poly1.touches(poly2)

def get_bounding_box_side(polygons):
    """Get the side length of the square bounding box."""
    if not polygons:
        return 0
    union = unary_union(polygons)
    bounds = union.bounds  # (minx, miny, maxx, maxy)
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def calculate_score(side_lengths):
    """Calculate total score from side lengths dict."""
    score = 0
    for n, side in side_lengths.items():
        score += side ** 2 / n
    return score

# Test
test_poly = create_tree_polygon(0, 0, 90)
print(f"Test polygon bounds: {test_poly.bounds}")
print(f"Test polygon area: {test_poly.area:.4f}")

Test polygon bounds: (-0.8, -0.35, 0.2, 0.35)
Test polygon area: 0.2456


In [3]:
def parse_submission(filepath):
    """Parse submission CSV into dict of configurations."""
    df = pd.read_csv(filepath)
    configs = defaultdict(list)
    
    for _, row in df.iterrows():
        id_parts = row['id'].split('_')
        n = int(id_parts[0])
        idx = int(id_parts[1])
        
        # Remove 's' prefix from values
        x = float(str(row['x'])[1:])
        y = float(str(row['y'])[1:])
        deg = float(str(row['deg'])[1:])
        
        configs[n].append((x, y, deg))
    
    return configs

# Load sample submission
sample_configs = parse_submission('/home/data/sample_submission.csv')
print(f"Loaded {len(sample_configs)} configurations (N=1 to N={max(sample_configs.keys())})")
print(f"N=1: {sample_configs[1]}")
print(f"N=5: {sample_configs[5]}")

Loaded 200 configurations (N=1 to N=200)
N=1: [(0.0, 0.0, 90.0)]
N=5: [(0.0, 0.0, 90.0), (0.202736, -0.511271, 90.0), (0.5206, 0.177413, 180.0), (-0.818657, -0.228694, 180.0), (0.111852, 0.893022, 180.0)]


In [4]:
def evaluate_config(trees_data):
    """Evaluate a configuration and return (side_length, has_overlaps)."""
    polygons = [create_tree_polygon(x, y, deg) for x, y, deg in trees_data]
    
    # Check for overlaps
    has_overlaps = False
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if has_overlap(polygons[i], polygons[j]):
                has_overlaps = True
                break
        if has_overlaps:
            break
    
    side = get_bounding_box_side(polygons)
    return side, has_overlaps

# Evaluate sample submission
print("Evaluating sample submission...")
sample_sides = {}
overlap_count = 0

for n in range(1, 201):
    side, has_overlaps = evaluate_config(sample_configs[n])
    sample_sides[n] = side
    if has_overlaps:
        overlap_count += 1
        if overlap_count <= 5:
            print(f"  N={n}: OVERLAP detected!")

sample_score = calculate_score(sample_sides)
print(f"\nSample submission score: {sample_score:.6f}")
print(f"Configurations with overlaps: {overlap_count}")

Evaluating sample submission...



Sample submission score: 173.652299
Configurations with overlaps: 0


In [5]:
# Greedy placement algorithm with weighted angles
def greedy_place_trees(n_trees, num_attempts=10, seed=None):
    """Place n trees using greedy algorithm with weighted random angles."""
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    
    best_config = None
    best_side = float('inf')
    
    for attempt in range(num_attempts):
        trees = []
        polygons = []
        
        # Place first tree at origin with random angle
        angle = random.uniform(0, 360)
        trees.append((0, 0, angle))
        polygons.append(create_tree_polygon(0, 0, angle))
        
        for i in range(1, n_trees):
            # Try multiple angles with weighted distribution (sin(2*angle) favors diagonals)
            best_tree = None
            best_tree_side = float('inf')
            
            for _ in range(20):  # Try 20 random angles
                # Weighted angle selection
                angle = random.uniform(0, 360)
                weight = abs(math.sin(2 * math.radians(angle)))
                if random.random() > weight:
                    continue
                
                # Find placement by moving from far away toward center
                placed = False
                for dist in np.linspace(10, 0.1, 50):
                    for theta in np.linspace(0, 2*np.pi, 36, endpoint=False):
                        cx = dist * np.cos(theta)
                        cy = dist * np.sin(theta)
                        
                        new_poly = create_tree_polygon(cx, cy, angle)
                        
                        # Check for overlaps
                        overlap = False
                        for existing_poly in polygons:
                            if has_overlap(new_poly, existing_poly):
                                overlap = True
                                break
                        
                        if not overlap:
                            test_polygons = polygons + [new_poly]
                            side = get_bounding_box_side(test_polygons)
                            if side < best_tree_side:
                                best_tree_side = side
                                best_tree = (cx, cy, angle, new_poly)
                            placed = True
                            break
                    if placed:
                        break
            
            if best_tree is not None:
                trees.append((best_tree[0], best_tree[1], best_tree[2]))
                polygons.append(best_tree[3])
            else:
                # Fallback: place at a safe distance
                angle = random.uniform(0, 360)
                dist = 5 + i * 0.5
                theta = random.uniform(0, 2*np.pi)
                cx, cy = dist * np.cos(theta), dist * np.sin(theta)
                trees.append((cx, cy, angle))
                polygons.append(create_tree_polygon(cx, cy, angle))
        
        side = get_bounding_box_side(polygons)
        if side < best_side:
            best_side = side
            best_config = trees
    
    return best_config, best_side

# Test greedy placement for small N
print("Testing greedy placement...")
for n in [2, 3, 5, 10]:
    config, side = greedy_place_trees(n, num_attempts=3, seed=42)
    sample_side = sample_sides[n]
    print(f"N={n}: greedy={side:.4f}, sample={sample_side:.4f}, diff={side-sample_side:.4f}")

Testing greedy placement...
N=2: greedy=10.4340, sample=1.2113, diff=9.2227
N=3: greedy=10.4340, sample=1.6706, diff=8.7634
N=5: greedy=10.4199, sample=2.1217, diff=8.2982


N=10: greedy=10.4178, sample=3.4411, diff=6.9767


In [ ]:
# Local search optimization
def local_search_optimize(trees_data, max_iterations=100, step_sizes=[0.02, 0.008, 0.003, 0.001]):
    """Optimize tree positions using local search."""
    trees = list(trees_data)
    polygons = [create_tree_polygon(x, y, deg) for x, y, deg in trees]
    current_side = get_bounding_box_side(polygons)
    
    improved = True
    iteration = 0
    
    while improved and iteration < max_iterations:
        improved = False
        iteration += 1
        
        for i in range(len(trees)):
            x, y, deg = trees[i]
            
            # Try different step sizes
            for step in step_sizes:
                # 8-directional moves
                for dx, dy in [(step, 0), (-step, 0), (0, step), (0, -step),
                               (step, step), (step, -step), (-step, step), (-step, -step)]:
                    new_x, new_y = x + dx, y + dy
                    new_poly = create_tree_polygon(new_x, new_y, deg)
                    
                    # Check for overlaps with other trees
                    overlap = False
                    for j, other_poly in enumerate(polygons):
                        if i != j and has_overlap(new_poly, other_poly):
                            overlap = True
                            break
                    
                    if not overlap:
                        test_polygons = polygons[:i] + [new_poly] + polygons[i+1:]
                        new_side = get_bounding_box_side(test_polygons)
                        
                        if new_side < current_side - 1e-8:
                            trees[i] = (new_x, new_y, deg)
                            polygons[i] = new_poly
                            current_side = new_side
                            improved = True
            
            # Try rotation adjustments
            for d_deg in [5, -5, 2, -2, 0.8, -0.8, 0.3, -0.3]:
                new_deg = (deg + d_deg) % 360
                new_poly = create_tree_polygon(x, y, new_deg)
                
                overlap = False
                for j, other_poly in enumerate(polygons):
                    if i != j and has_overlap(new_poly, other_poly):
                        overlap = True
                        break
                
                if not overlap:
                    test_polygons = polygons[:i] + [new_poly] + polygons[i+1:]
                    new_side = get_bounding_box_side(test_polygons)
                    
                    if new_side < current_side - 1e-8:
                        trees[i] = (x, y, new_deg)
                        polygons[i] = new_poly
                        current_side = new_side
                        improved = True
    
    return trees, current_side

# Test local search on sample config
print("Testing local search optimization...")
for n in [5, 10, 20]:
    original_side = sample_sides[n]
    optimized, new_side = local_search_optimize(sample_configs[n], max_iterations=20)
    print(f"N={n}: original={original_side:.4f}, optimized={new_side:.4f}, improvement={original_side-new_side:.4f}")

In [ ]:
# Squeeze/compaction toward centroid
def squeeze_config(trees_data, squeeze_factor=0.99, max_iterations=100):
    """Squeeze trees toward centroid until overlap occurs."""
    trees = list(trees_data)
    
    for iteration in range(max_iterations):
        # Calculate centroid
        cx = sum(t[0] for t in trees) / len(trees)
        cy = sum(t[1] for t in trees) / len(trees)
        
        # Try to squeeze all trees toward centroid
        new_trees = []
        for x, y, deg in trees:
            new_x = cx + (x - cx) * squeeze_factor
            new_y = cy + (y - cy) * squeeze_factor
            new_trees.append((new_x, new_y, deg))
        
        # Check for overlaps
        polygons = [create_tree_polygon(x, y, deg) for x, y, deg in new_trees]
        has_overlaps = False
        for i in range(len(polygons)):
            for j in range(i+1, len(polygons)):
                if has_overlap(polygons[i], polygons[j]):
                    has_overlaps = True
                    break
            if has_overlaps:
                break
        
        if has_overlaps:
            break
        
        trees = new_trees
    
    return trees, get_bounding_box_side([create_tree_polygon(x, y, deg) for x, y, deg in trees])

# Test squeeze
print("Testing squeeze optimization...")
for n in [10, 20, 50]:
    original_side = sample_sides[n]
    squeezed, new_side = squeeze_config(sample_configs[n])
    print(f"N={n}: original={original_side:.4f}, squeezed={new_side:.4f}, improvement={original_side-new_side:.4f}")

In [ ]:
# Combined optimization pipeline
def optimize_config(trees_data, time_limit=5.0):
    """Full optimization pipeline for a single configuration."""
    start_time = time.time()
    
    trees = list(trees_data)
    polygons = [create_tree_polygon(x, y, deg) for x, y, deg in trees]
    best_side = get_bounding_box_side(polygons)
    best_trees = trees
    
    # Phase 1: Squeeze
    squeezed, side = squeeze_config(trees)
    if side < best_side:
        best_side = side
        best_trees = squeezed
    
    # Phase 2: Local search
    while time.time() - start_time < time_limit:
        optimized, side = local_search_optimize(best_trees, max_iterations=10)
        if side < best_side - 1e-8:
            best_side = side
            best_trees = optimized
        else:
            break
    
    # Phase 3: Final squeeze
    squeezed, side = squeeze_config(best_trees)
    if side < best_side:
        best_side = side
        best_trees = squeezed
    
    return best_trees, best_side

# Test combined optimization
print("Testing combined optimization...")
for n in [10, 20, 50, 100]:
    original_side = sample_sides[n]
    optimized, new_side = optimize_config(sample_configs[n], time_limit=3.0)
    print(f"N={n}: original={original_side:.4f}, optimized={new_side:.4f}, improvement={original_side-new_side:.4f}")

In [ ]:
# Optimize all configurations
print("Optimizing all configurations...")
print("This will take a while...")

optimized_configs = {}
optimized_sides = {}

start_time = time.time()

for n in range(1, 201):
    # Use sample config as starting point
    original_config = sample_configs[n]
    original_side = sample_sides[n]
    
    # Optimize with time proportional to N (larger N gets more time)
    time_limit = 0.5 + n * 0.02  # 0.5s to 4.5s per config
    
    optimized, new_side = optimize_config(original_config, time_limit=time_limit)
    
    # Keep better result
    if new_side < original_side:
        optimized_configs[n] = optimized
        optimized_sides[n] = new_side
    else:
        optimized_configs[n] = original_config
        optimized_sides[n] = original_side
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        current_score = calculate_score(optimized_sides)
        print(f"N={n}: score so far = {current_score:.6f}, elapsed = {elapsed:.1f}s")

total_time = time.time() - start_time
final_score = calculate_score(optimized_sides)
print(f"\nOptimization complete in {total_time:.1f}s")
print(f"Final score: {final_score:.6f}")
print(f"Sample score: {sample_score:.6f}")
print(f"Improvement: {sample_score - final_score:.6f}")

In [ ]:
# Validate no overlaps
print("Validating configurations...")
overlap_count = 0
for n in range(1, 201):
    _, has_overlaps = evaluate_config(optimized_configs[n])
    if has_overlaps:
        overlap_count += 1
        print(f"  N={n}: OVERLAP detected!")

print(f"\nConfigurations with overlaps: {overlap_count}")
if overlap_count > 0:
    print("WARNING: Some configurations have overlaps - submission may fail!")

In [ ]:
# Create submission file
def create_submission(configs, filepath):
    """Create submission CSV from configurations dict."""
    rows = []
    for n in range(1, 201):
        for idx, (x, y, deg) in enumerate(configs[n]):
            rows.append({
                'id': f'{n:03d}_{idx}',
                'x': f's{x:.6f}',
                'y': f's{y:.6f}',
                'deg': f's{deg:.6f}'
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filepath, index=False)
    print(f"Saved submission to {filepath}")
    return df

# Save submission
submission_df = create_submission(optimized_configs, '/home/submission/submission.csv')
print(f"\nSubmission shape: {submission_df.shape}")
print(submission_df.head(10))

In [ ]:
# Summary
print("="*60)
print("BASELINE EXPERIMENT SUMMARY")
print("="*60)
print(f"Sample submission score: {sample_score:.6f}")
print(f"Optimized score: {final_score:.6f}")
print(f"Improvement: {sample_score - final_score:.6f}")
print(f"Configurations with overlaps: {overlap_count}")
print("="*60)